In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Running SQL Queries for IMDb") \
    .getOrCreate()

23/10/20 15:11:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
df1 = spark.read.csv("ImdbName.csv", header = True)
df1.createOrReplaceTempView("Name")

df2 = spark.read.csv("ImdbTitleAkas.csv", header = True)
df2.createOrReplaceTempView("TitleAkas")

df3 = spark.read.csv("ImdbTitleBasics.csv", header = True)
df3.createOrReplaceTempView("TitleBasics")

df4 = spark.read.csv("ImdbTitleCrew.csv", header = True)
df4.createOrReplaceTempView("TitleCrew")

df5 = spark.read.csv("ImdbTitleEpisode.csv", header = True)
df5.createOrReplaceTempView("TitleEpisode")

df6 = spark.read.csv("ImdbTitlePrincipals.csv", header = True)
df6.createOrReplaceTempView("TitlePrincipals")

df7 = spark.read.csv("ImdbTitleRatings.csv", header = True)
df7.createOrReplaceTempView("TitleRatings")

In [15]:
# df1.show()

In [17]:
path = "queries/1.sql"
with open(path) as sql_file:
    sql_query = sql_file.read()
result = spark.sql(sql_query)
result.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy